In [1]:
import pandas as pd
import numpy as np
from numpy import arange
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

from cm import plot_confusion_matrix

import tensorflow as tf

In [2]:
real_codes = pd.read_csv('data/J1939Faults.csv', low_memory = False)
onboard = pd.read_csv('data/VehicleDiagnosticOnboardData.csv', low_memory = False)

# Initial data cleaning and filtering

In [3]:
#Per the project's instructions: 

real_codes = real_codes[real_codes['EquipmentID'].str.len() < 6]
real_codes = real_codes[real_codes['ecuMake'] == 'CMMNS']
real_codes = real_codes[real_codes['active'] == True]

In [4]:
real_codes = real_codes.drop(columns=['actionDescription', 'faultValue', 'MCTNumber'])

In [5]:
real_codes[real_codes['spn'] == 3362]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp
11438,12321,1514613,2015-03-24 11:07:15.000,Condition Exists Catalyst Dosing Unit Input Lines,04993120*00019058*082113134117*07700053*I0*BBZ*,79461375,6X1u10D1500000000,CMMNS,0,3362,31,True,1,1370,36.876574,-81.397361,2015-03-24 11:07:51.000
11475,12358,1519318,2015-03-24 14:37:17.000,Condition Exists Catalyst Dosing Unit Input Lines,04993120*00019058*082113134117*07700053*I0*BBZ*,79461375,6X1u10D1500000000,CMMNS,0,3362,31,True,1,1370,36.194861,-83.174768,2015-03-24 14:37:52.000
11491,12374,1520761,2015-03-24 15:41:08.000,Condition Exists Catalyst Dosing Unit Input Lines,04993120*00019058*082113134117*07700053*I0*BBZ*,79461375,6X1u10D1500000000,CMMNS,0,3362,31,True,1,1370,36.194814,-83.174722,2015-03-24 15:41:45.000
11646,12529,1531190,2015-03-25 07:58:21.000,Condition Exists Catalyst Dosing Unit Input Lines,04993120*00019058*082113134117*07700053*I0*BBZ*,79461375,6X1u10D1500000000,CMMNS,0,3362,31,True,1,1370,36.194907,-83.175000,2015-03-25 07:58:58.000
17240,18123,1816124,2015-04-10 10:08:09.000,Condition Exists Catalyst Dosing Unit Input Lines,05317106*04260187*092514204317*09400024*G1*BDR*,79813609,6X1u13D1500000000,CMMNS,0,3362,31,True,1,1785,36.174722,-86.022592,2015-04-10 10:08:46.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176634,1236775,118452393,2020-01-26 05:02:23.000,Condition Exists Catalyst Dosing Unit Input Lines,04358814*06107315*061516161145*09401661*G1*BDR*,79931760,6X1u13D1500000000,CMMNS,0,3362,31,True,1,1991,35.840787,-86.428287,2020-01-26 05:02:58.000
1181915,1242056,120991271,2020-02-14 07:27:54.000,Condition Exists Catalyst Dosing Unit Input Lines,04358814*06011766*122016144238*09401671*G1*BDR*,79897856,6X1u13D1500000000,CMMNS,0,3362,31,True,1,1862,40.437129,-75.908657,2020-02-14 07:28:30.000
1182447,1242588,121281871,2020-02-17 05:38:19.000,Condition Exists Catalyst Dosing Unit Input Lines,05317106*05031273*050815201656*09400035*G1*BDR*,79857685,6X1u13D1500000000,CMMNS,0,3362,31,True,1,1817,35.607453,-82.329027,2020-02-17 05:38:55.000
1182512,1242653,121330201,2020-02-17 10:44:39.000,Condition Exists Catalyst Dosing Unit Input Lines,05317106*05005224*051718172255*09401583*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,3362,31,True,1,1814,35.828333,-86.414629,2020-02-17 10:45:15.000


In [6]:
onboard

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
...,...,...,...,...
12821621,12864020,EngineCoolantTemperature,181.4,1248457
12821622,12864021,ParkingBrake,False,1248457
12821623,12864022,SwitchedBatteryVoltage,14.1,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457


In [9]:
#Selects lat and lon coordinates that are not in the bounded boxes

geo_codes = real_codes[(~real_codes['Latitude'].between(36.05942, 36.07392)) | (~real_codes['Longitude'].between(-86.44366, -86.42579))]
geo_codes = geo_codes[(~geo_codes['Latitude'].between(35.58108, 35.59558)) | (~geo_codes['Longitude'].between(-86.4528, -86.435))]
geo_codes = geo_codes[(~geo_codes['Latitude'].between(36.18775, 36.20225)) | (~geo_codes['Longitude'].between(-83.1837, -83.1658))]

#Here is an alternative method for selecting lats and lons that are not in the bounded boxes: 

#geo_codes = real_codes.query('(Latitude <= 36.05942 or Latitude >= 36.07392) or (Longitude <= -86.5419 or Longitude >= -86.3275)')


In [10]:
geo_codes = geo_codes.sort_values(["EquipmentID", "EventTimeStamp"], ascending = (False, True))

# Some EDA

In [11]:
#Number of unique trucks in the data

geo_codes.EquipmentID.nunique()

928

In [12]:
geo_codes_select1569 = geo_codes[geo_codes['spn'].isin([1569])]
geo_codes_select5246 = geo_codes[geo_codes['spn'].isin([5246])]

In [13]:
#Number of unique 1569s thrown by the ECUs

geo_codes_select1569['RecordID'].nunique()

4277

In [14]:
#Number of unique 5426s thrown by the ECUs

geo_codes_select5246['RecordID'].nunique()

270

In [15]:
geo_codes.RecordID.nunique(())

200154

In [16]:
#Looking at the names of the metrics that are recorded when the ECU throws a code

geo_codes[geo_codes['RecordID']  == 4246].head(25)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp
4245,4246,1048627,2015-02-24 13:45:06.000,Low (Severity Low) Catalyst Tank Level,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,0,1761,17,True,43,R1762,41.254166,-85.088888,2015-02-24 13:45:41.000


In [17]:
geo_codes.eventDescription.value_counts()

Low (Severity Low) Engine Coolant Level                                         137206
Low (Severity Medium) Engine Coolant Level                                       14676
High (Severity Low) Water In Fuel Indicator                                       6992
Condition Exists Engine Protection Torque Derate                                  4277
High Voltage (Water In Fuel Indicator)                                            3768
                                                                                 ...  
High Voltage (Engine Oil Pressure)                                                   1
Abnormal Update Rate Engine Wait to Start Lamp                                       1
High (Severity Medium) Engine Exhaust Gas Recirculation (EGR) Mass Flow Rate         1
High Current Engine Exhaust Gas Recirculation (EGR) Valve Control                    1
High Voltage (Barometric Pressure)                                                   1
Name: eventDescription, Length: 266, dtype:

# More data cleaning and filtering based on prior EDA

In [18]:
geo_codes = geo_codes.drop(columns=['LocationTimeStamp', 'active', 'ecuSource', 'ecuMake', 'ESS_Id'])

In [19]:
#Reorders our columns in a logical and readable way: index types, data types, others

geo_codes = geo_codes.reindex(columns = ['RecordID', 'EquipmentID', 'EventTimeStamp', 'spn', 'fmi', 'spnfmi',
                                         'activeTransitionCount', 'ecuModel', 'ecuSerialNumber', 'ecuSoftwareVersion',
                                         'eventDescription', 'Latitude', 'Longitude'])

In [20]:
#Concats the spn and fmi fields into a single field, then converts them back to an int

geo_codes['spnfmi'] = geo_codes.spn.astype('str').map(str) + geo_codes.fmi.astype('str')
geo_codes['spnfmi'] = geo_codes['spnfmi'].astype('int32')

# Get dummy variables in preperation of logistic regression

In [21]:
geo_codes_spnfmi = pd.get_dummies(geo_codes, columns = ['spnfmi'])

# Convert datatime column for indexing and sorting

In [22]:
geo_codes_spnfmi['EventTimeStamp'] = geo_codes_spnfmi['EventTimeStamp'].astype('datetime64[ns]')

In [23]:
geo_codes_spnfmi['EventTimeStamp'] = pd.to_datetime(geo_codes_spnfmi['EventTimeStamp'], utc = True)

In [24]:
geo_codes_dropped = geo_codes_spnfmi.reset_index()

In [25]:
#List comprehension of columns created from the get_dummies

List = [x for x in geo_codes_dropped.columns if 'spnfmi_' in x]

# Pandas .rolling() to create a cumulative count of spn-fmi combo codes thrown

In [26]:
geo_codes_dropped = geo_codes_dropped.sort_values(['EquipmentID', 'EventTimeStamp']).reset_index(drop = True)

In [27]:
geo_codes_dropped_rolling = geo_codes_dropped.groupby('EquipmentID').rolling(window = "14d", on = 'EventTimeStamp')[List].sum().reset_index()

In [298]:
merge_geo_codes = geo_codes_dropped.merge(geo_codes_dropped_rolling, left_index = True, right_index = True)

In [299]:
merge_geo_codes = merge_geo_codes.rename(columns = {'EquipmentID_x' : 'EquipmentID', 'EventTimeStamp_x' : 'EventTimeStamp'})

In [300]:
merge_geo_codes = merge_geo_codes.loc[:,~merge_geo_codes.columns.str.contains('_x', case=False)]

In [301]:
merge_geo_codes = merge_geo_codes.drop(columns=['index'])

In [302]:
#Saved for future use when coming back and merging onboard

In [303]:
def backfill (df):
    df['flagtime'] = df.loc[df['spnfmi_156931_y'] == 1, 'EventTimeStamp']
    df['flagtime'] = df.flagtime.bfill()
    return df

In [304]:
merge_geo_codes = merge_geo_codes.groupby('EquipmentID').apply(backfill)

In [305]:
merge_geo_codes['deltatime'] = merge_geo_codes['flagtime'] - merge_geo_codes['EventTimeStamp']
close_codes = merge_geo_codes

In [306]:
close_codes['targettime'] = merge_geo_codes.deltatime.dt.total_seconds().between(-1, 3600)
merge_geo_codes['targettime'] = merge_geo_codes.deltatime.dt.total_seconds().between(3600, 604800)


In [307]:
merge_geo_codes = merge_geo_codes[~merge_geo_codes['deltatime'].dt.total_seconds().between(-1, 3600)]
close_codes = close_codes[close_codes['deltatime'].dt.total_seconds().between(-1, 3600)]

In [308]:
merge_geo_codes = merge_geo_codes[merge_geo_codes['spn'] != 5246]
close_codes = close_codes[close_codes['spn'] != 5246]

In [309]:
#merge_geo_codes.to_csv('data/merge_geo_codes2.csv', index = False)

In [310]:
onboard_drop = onboard[['FaultId', 'Name', 'Value']]

In [311]:
onboard_ltd = onboard_drop.query('Name == "DistanceLtd" or Name == "EngineTimeLtd"')

In [312]:
onboard_pivot = onboard_ltd.pivot(index = 'FaultId', columns = 'Name', values = 'Value')

In [313]:
onboard_pivot.reset_index(inplace = True)

In [352]:
good_trucks = merge_geo_codes.merge(onboard_pivot, how = 'left', left_on = 'RecordID', right_on = 'FaultId')
bad_trucks = close_codes.merge(onboard_pivot, how = 'left', left_on = 'RecordID', right_on = 'FaultId')

In [353]:
good_trucks = good_trucks[['RecordID', 'EquipmentID', 'spn', 'fmi', 'eventDescription', 'DistanceLtd', 
                           'EngineTimeLtd', 'flagtime', 'deltatime', 'targettime', 'EventTimeStamp']]
bad_trucks = bad_trucks[['RecordID', 'EquipmentID', 'spn', 'fmi', 'eventDescription', 'DistanceLtd', 
                           'EngineTimeLtd', 'flagtime', 'deltatime', 'targettime', 'EventTimeStamp']]

In [354]:
good_trucks[['EngineTimeLtd', 'DistanceLtd']] = good_trucks[['EngineTimeLtd', 'DistanceLtd']].apply(pd.to_numeric)
bad_trucks[['EngineTimeLtd', 'DistanceLtd']] = bad_trucks[['EngineTimeLtd', 'DistanceLtd']].apply(pd.to_numeric)

In [355]:
good_trucks = good_trucks[good_trucks['targettime'] == True].sort_values('DistanceLtd', ascending = True)
bad_trucks = bad_trucks[bad_trucks['targettime'] == True].sort_values('DistanceLtd', ascending = True)

In [356]:
good_trucks

,RecordID,EquipmentID,spn,fmi,eventDescription,DistanceLtd,EngineTimeLtd,flagtime,deltatime,targettime,EventTimeStamp
186892,679467,2017,111,18,Low (Severity Medium) Engine Coolant Level,154.3020,10.85,2017-01-12 10:02:45+00:00,6 days 06:17:08,True,2017-01-06 03:45:37+00:00
186893,679619,2017,111,18,Low (Severity Medium) Engine Coolant Level,353.8367,14.00,2017-01-12 10:02:45+00:00,6 days 03:02:35,True,2017-01-06 07:00:10+00:00
186894,680431,2017,111,18,Low (Severity Medium) Engine Coolant Level,447.5022,16.55,2017-01-12 10:02:45+00:00,5 days 02:36:46,True,2017-01-07 07:25:59+00:00
130190,27820,1789,111,17,Low (Severity Low) Engine Coolant Level,534.8981,13.30,2015-04-23 16:52:41+00:00,3 days 06:20:07,True,2015-04-20 10:32:34+00:00
130191,27877,1789,111,17,Low (Severity Low) Engine Coolant Level,576.1757,14.00,2015-04-23 16:52:41+00:00,3 days 05:39:00,True,2015-04-20 11:13:41+00:00
...,...,...,...,...,...,...,...,...,...,...,...
129364,186994,1784,3464,7,Not Reporting Data Engine Throttle Actuator 1 ...,NaN,NaN,2015-09-04 17:31:12+00:00,5 days 00:43:12,True,2015-08-30 16:48:00+00:00
130196,28730,1789,111,17,Low (Severity Low) Engine Coolant Level,NaN,NaN,2015-04-23 16:52:41+00:00,2 days 10:33:36,True,2015-04-21 06:19:05+00:00
130260,131671,1790,1209,2,Incorrect Data Engine Exhaust Gas Pressure,NaN,NaN,2015-07-22 16:33:01+00:00,0 days 18:29:17,True,2015-07-21 22:03:44+00:00
130261,131672,1790,5019,2,NaN,NaN,NaN,2015-07-22 16:33:01+00:00,0 days 18:29:17,True,2015-07-21 22:03:44+00:00


In [360]:
good_trucks[['RecordID', 'spn', 'DistanceLtd', 'EngineTimeLtd', 'flagtime']].groupby('flagtime').min().describe()

,RecordID,spn,DistanceLtd,EngineTimeLtd
count,9.040000e+02,904.000000,896.000000,894.000000
mean,7.310704e+05,1713.234513,347141.235995,7158.070861
std,3.155253e+05,1562.328853,159993.817701,4891.046047
min,3.640000e+02,27.000000,154.302000,10.850000
25%,4.979052e+05,111.000000,221987.800000,4305.575000
50%,7.568910e+05,1761.000000,373516.300000,7551.425000
75%,9.979680e+05,3251.000000,483566.650000,9967.662500
max,1.246248e+06,6802.000000,627421.900000,112054.200000


In [361]:
bad_trucks[['RecordID', 'spn', 'DistanceLtd', 'EngineTimeLtd', 'flagtime']].groupby('flagtime').min().describe()

,RecordID,spn,DistanceLtd,EngineTimeLtd
count,7.600000e+01,76.000000,76.000000,76.000000
mean,8.576269e+05,3857.526316,291176.211908,5834.527632
std,2.525185e+05,1015.008210,163724.355406,3405.250408
min,2.363440e+05,1761.000000,2101.185000,68.850000
25%,7.043618e+05,3362.000000,170601.400000,3110.887500
50%,9.425795e+05,3362.000000,295516.300000,5916.675000
75%,1.042182e+06,4094.000000,417004.100000,8479.637500
max,1.230667e+06,6802.000000,593736.800000,12697.400000


In [322]:
#merge_geo_codes[merge_geo_codes['spn'] == 3362]

# Create a modified version of a train test split

In [38]:
#A function that simulates a train test split that keeps time sequences for trucks together, but still allows individual trucks
#to randomly be assigned to training and testing

def train_test_brake (df):
    df = df.sort_values('EventTimeStamp')
    t_len = df.shape[0]
    t_perc80 = int(t_len * 0.8)
    df80 = df.iloc[0 : t_perc80, :]
    df20 = df.iloc[t_perc80 : , :]
    return(df80, df20) 

In [39]:
dataframe_80_20 = merge_geo_codes.groupby('EquipmentID').apply(train_test_brake)

In [40]:
#Creates training dataset

df80 = pd.concat([x[0] for x in dataframe_80_20])

In [41]:
#Creates testing dataset

df20 = pd.concat([x[1] for x in dataframe_80_20])

# Logistic Regression

#predictors = df80.columns.tolist()
bad_predictors = ['RecordID', 'EquipmentID', 'EventTimeStamp', 
                  'spn', 'fmi', 'ecuModel', 'activeTransitionCount',
                  'ecuSerialNumber', 'ecuSoftwareVersion', 
                  'eventDescription', 'Latitude', 'Longitude', 
                  'EquipmentID_y', 'EventTimeStamp_y', 'spnfmi_156931_y', 
                  'flagtime', 'deltatime', 'targettime']

In [42]:
#predictors.remove(bad_predictors)

predictors_train = df80.drop(bad_predictors, axis = 1)
predictors_test = df20.drop(bad_predictors, axis = 1)

#df20 = df20.drop(bad_predictors, axis = 1)

predictors_train

# Predictors
X_train = predictors_train

X_test = predictors_test

# Target
y_train = df80.targettime

y_test = df20.targettime

logreg = LogisticRegression().fit(X_train, y_train)

lr_weights = LogisticRegression(class_weight = {0:1, 1:12})

lr_weights.fit(X_train, y_train)

y_pred = lr_weights.predict(X_test)

accuracy_score(y_test, y_pred)

logreg.coef_

logreg.intercept_

y_pred = logreg.predict(X_test)

y_pred

logreg.predict_proba(X_test)

confusion_matrix(y_test, y_pred)

plot_confusion_matrix(y_test, y_pred, labels = ['0', '1'], metric = 'accuracy')

plot_confusion_matrix(y_test, y_pred, labels = ['0', '1'], metric = 'precision')

f1_score(y_test, y_pred, average = 'micro')

print(classification_report(y_test, y_pred))

# LASSO TESTING

In [43]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [44]:
#with tf.device('/gpu:0'):

#Smaller c values specify stronger regularization 
with tf.device('/gpu:0'):
    lasso = LogisticRegression(penalty = 'l1', 
                           C = 0.05, 
                           solver = 'saga', 
                           class_weight = 'balanced', 
                           max_iter = 1000, 
                           ).fit(X_train, y_train)

lr = LogisticRegression(penalty = 'l1', 
                       solver = 'saga', 
                       class_weight = 'balanced',
                       max_iter = 5000)

In [45]:
#param_grid = dict()

In [46]:
#param_grid['C'] = arange(0, 0.2, 0.05) 

In [47]:
#gs = GridSearchCV(estimator = lr, 
#                 param_grid = param_grid, 
#                 scoring = f1, 
#                 cv = 3)

with tf.device('/gpu:0'):
    results = gs.fit(X_train, y_train)

print(results.best_score_)

print(results.best_params_)

results